# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
****

## Import Data and Basic Information

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.stats.api as sms

/Users/DylanQian/anaconda/lib/python2.7/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='b'].call)

157.0

In [4]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [5]:
data.columns

Index([u'id', u'ad', u'education', u'ofjobs', u'yearsexp', u'honors',
       u'volunteer', u'military', u'empholes', u'occupspecific', u'occupbroad',
       u'workinschool', u'email', u'computerskills', u'specialskills',
       u'firstname', u'sex', u'race', u'h', u'l', u'call', u'city', u'kind',
       u'adid', u'fracblack', u'fracwhite', u'lmedhhinc', u'fracdropout',
       u'fraccolp', u'linc', u'col', u'expminreq', u'schoolreq', u'eoe',
       u'parent_sales', u'parent_emp', u'branch_sales', u'branch_emp', u'fed',
       u'fracblack_empzip', u'fracwhite_empzip', u'lmedhhinc_empzip',
       u'fracdropout_empzip', u'fraccolp_empzip', u'linc_empzip', u'manager',
       u'supervisor', u'secretary', u'offsupport', u'salesrep', u'retailsales',
       u'req', u'expreq', u'comreq', u'educreq', u'compreq', u'orgreq',
       u'manuf', u'transcom', u'bankreal', u'trade', u'busservice',
       u'othservice', u'missind', u'ownership'],
      dtype='object')

In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4870 entries, 0 to 4869
Data columns (total 65 columns):
id                    4870 non-null object
ad                    4870 non-null object
education             4870 non-null int8
ofjobs                4870 non-null int8
yearsexp              4870 non-null int8
honors                4870 non-null int8
volunteer             4870 non-null int8
military              4870 non-null int8
empholes              4870 non-null int8
occupspecific         4870 non-null int16
occupbroad            4870 non-null int8
workinschool          4870 non-null int8
email                 4870 non-null int8
computerskills        4870 non-null int8
specialskills         4870 non-null int8
firstname             4870 non-null object
sex                   4870 non-null object
race                  4870 non-null object
h                     4870 non-null float32
l                     4870 non-null float32
call                  4870 non-null float32
city        

In [7]:
data.describe()

,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,occupbroad,workinschool,...,educreq,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind
count,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,...,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000,4870.000000
mean,3.618480,3.661396,7.842916,0.052772,0.411499,0.097125,0.448049,215.637782,3.481520,0.559548,...,0.106776,0.437166,0.072690,0.082957,0.030390,0.085010,0.213963,0.267762,0.154825,0.165092
std,0.714997,1.219126,5.044612,0.223601,0.492156,0.296159,0.497345,148.127551,2.038036,0.496492,...,0.308866,0.496083,0.259649,0.275854,0.171677,0.278932,0.410141,0.442847,0.361773,0.371308
min,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,7.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,3.000000,5.000000,0.000000,0.000000,0.000000,0.000000,27.000000,1.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,4.000000,4.000000,6.000000,0.000000,0.000000,0.000000,0.000000,267.000000,4.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.000000,4.000000,9.000000,0.000000,1.000000,0.000000,1.000000,313.000000,6.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
max,4.000000,7.000000,44.000000,1.000000,1.000000,1.000000,1.000000,903.000000,6.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


First, I extract the callbacks from the datasets based on applicants race. 

In [8]:
black_callback = data.call[data.race=='b']
white_callback = data.call[data.race=='w']

In [9]:
black_n = black_callback.count()
black_n

2435

In [10]:
white_n = white_callback.count()
white_n

2435

## 1. What test is appropriate for this problem? Does CLT apply?

In [11]:
black_rate = np.sum(black_callback)/len(black_callback)
print(black_rate)
white_rate = np.sum(white_callback)/len(white_callback)
print(white_rate)

0.064476386037
0.0965092402464


In [12]:
empirical_diff_means = white_rate - black_rate
empirical_diff_means

0.032032854209445585

With 2435 observations for each group, black applicants has a 6.44% success rate to get a callback while white applicants has a 9.65% success rate, which makes a difference of 3.20%. Thus, I suspect that the race can influence the decision for callbacks.

In order for the result of the CLT to hold, the sample must be sufficiently large (n > 30). Here we have 2435 observations for each group, which is way beyond 30. Thus, in this case, CLT can apply. 

Additionally, there are two independent datasets, so I consider to use two-sample test to compare if race has a determining factor for callbacks.. 

## 2. What are the null and alternate hypotheses?

Null hypothesis: race (white or black) does not affect the decision on callbacks.

Alternative hypothesis: race does affect the decision on callbacks. 


## 3. Compute margin of error, confidence interval, and p-value.

First, I used the permutation test to resample the dataset and then get the p value. 

### Permutation Test

In [13]:
bs_replicates_race = np.empty(10000)
for i in range(10000):
    bs_sample = np.random.choice(data.call, size = len(data.call))
    bs_replicates_race[i] = np.mean(bs_sample[:len(white_callback)]) - np.mean(bs_sample[len(white_callback):])
bs_replicates_race

array([-0.0110883 ,  0.01806982,  0.00082135, ...,  0.01314168,
       -0.01190965, -0.00205339])

In [14]:
p =np.sum(bs_replicates_race >= empirical_diff_means)/len(bs_replicates_race)
p

0

### t-test

Also, I decide to use t-test method with scipy.stats in order to further test if the permutation test is right. 

In [15]:
stats.ttest_ind(black_callback, white_callback)

Ttest_indResult(statistic=-4.1147052908617514, pvalue=3.9408021031288859e-05)

From the result, we can see that the t-score is -4.115, and p-value is 0.0000394, which is nearly 0. Thus, it shows that the permutation test was correct. 

### z-test

In [16]:
white_std = np.std(white_callback)
black_std = np.std(black_callback)
print white_std, black_std

0.295284867287 0.245599016547


Because the number of samples for both group is the same, so we can easily add their variance together and divide by 2 to get their average variance.

In [17]:
average_var = (white_std**2 +black_std**2)/2
average_var

0.0737560148887334

For 95% confidence level, the z-score(z) is 1.96, which can be found on z-score table. 

In [18]:
z = 1.96
margin_of_error = z * average_var
round(margin_of_error,3)


0.145

In [19]:
cm = sms.CompareMeans(sms.DescrStatsW(white_callback), sms.DescrStatsW(black_callback))
print cm.tconfint_diff(usevar='unequal')

(0.016770673983991798, 0.047295034434899373)


Thus, we get our confidence interval is 0.016 and 0.047, and the margin or errors is 0.145. We got the p-value from last question already, which is 0.0000394 or 0. 

## 4. Write a story describing the statistical significance in the context or the original problem.

Because the p-value is 0.0000394, which is nearly 0, I have to reject null hypothesis. This means that there is a difference for callbacks for two different sample groups ,so race can be a factor that affect the decision on callback. Thus, it is possible that there is racial discrimination in the US job market that prevent black people to have a fair competition with white people. 


In this case, we believe that race can affect the decision making process for employments.  

## 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

From the analysis. I determine that race can significantly affect the decisions on callbacks. However,in this datasets, there are a lot more different variables I did not look into, and they may or may not have correlation with race, so race might not be the only reason that they did not get callbacks. Thus, I recommend furthur group to look into other variables. 
